<a href="https://colab.research.google.com/github/Janhvi-n/AlpaCare-Medical-Instruction-Assistant-/blob/main/AlpaCare_Medical_Instruction_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y transformers accelerate datasets peft
!pip install -U transformers==4.44.2 datasets accelerate peft -q

Found existing installation: transformers 4.57.0
Uninstalling transformers-4.57.0:
  Successfully uninstalled transformers-4.57.0
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
Found existing installation: datasets 4.1.1
Uninstalling datasets-4.1.1:
  Successfully uninstalled datasets-4.1.1
Found existing installation: peft 0.17.1
Uninstalling peft-0.17.1:
  Successfully uninstalled peft-0.17.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 110.7 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade transformers datasets peft accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [ ]:
import transformers
print(transformers.__version__)

4.57.0


In [ ]:
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq, TrainingArguments, Trainer)
from peft import LoraConfig, get_peft_model, TaskType
import torch

In [ ]:
# data_loader.py
from datasets import load_dataset

def load_and_preprocess():
    # Load dataset
    dataset = load_dataset("lavita/AlpaCare-MedInstruct-52k")

    # Use only a subset (saves Colab time)
    dataset = dataset["train"].shuffle(seed=42).select(range(2000))

    # Split into train/val/test (90/5/5)
    train_val = dataset.train_test_split(test_size=0.10, seed=42)
    val_test = train_val["test"].train_test_split(test_size=0.50, seed=42)

    train_data = train_val["train"]
    val_data = val_test["train"]
    test_data = val_test["test"]

    return train_data, val_data, test_data


In [ ]:
train_data, val_data, test_data = load_and_preprocess()

# Format function
def format_data(example):
    return {
        "input_text": example["instruction"],
        "target_text": example["output"]
    }

train_data = train_data.map(format_data)
val_data = val_data.map(format_data)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05
)
model = get_peft_model(model, lora_config)

In [ ]:
MAX_LENGTH = 128
def tokenize(batch):
    # Input
    model_inputs = tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
    # Target/labels
    labels = tokenizer(batch["target_text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
    labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_data = train_data.map(tokenize, batched=True)
val_data = val_data.map(tokenize, batched=True)

train_data.set_format(type="torch", columns=["input_ids","attention_mask","labels"])
val_data.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    eval_steps=100,
    evaluation_strategy="steps",
    learning_rate=2e-4,
    fp16=torch.cuda.is_available(),
    report_to="none"  # disables W&B, TensorBoard, etc.
)

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
100,0.000000,nan
200,0.000000,nan


TrainOutput(global_step=225, training_loss=0.0, metrics={'train_runtime': 119.7532, 'train_samples_per_second': 15.031, 'train_steps_per_second': 1.879, 'total_flos': 309363867648000.0, 'train_loss': 0.0, 'epoch': 1.0})

In [ ]:
model.save_pretrained("alpacare_lora_adapter")
tokenizer.save_pretrained("alpacare_lora_adapter")
print("✅ LoRA adapter saved successfully!")

✅ LoRA adapter saved successfully!


In [ ]:
# --------------------------
# 🔹 Inference Demo
# --------------------------
from peft import PeftModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load base model + LoRA adapter
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, "alpacare_lora_adapter")
model.to(device)
model.eval()

MAX_LENGTH = 128

def ask(prompt, max_new_tokens=150, temperature=0.7, top_p=0.9):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH
    )
    # Move tensors to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate output using sampling
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,      # enable sampling
        top_p=top_p,         # nucleus sampling
        temperature=temperature,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text + "\n\n⚠️ This is educational only — please consult a qualified clinician."

# --------------------------
# 🔹 Test prompts
# --------------------------
prompts_list = [
    "What are healthy food options for someone with diabetes?",
    "Explain how to properly use an inhaler.",
    "Describe safe hand-washing steps to prevent infection."
]

for p in prompts_list:
    print("Prompt:", p)
    print(ask(p))
    print("------------------------------------------------------\n")


Prompt: What are healthy food options for someone with diabetes?
sweet potatoes

⚠️ This is educational only — please consult a qualified clinician.
------------------------------------------------------

Prompt: Explain how to properly use an inhaler.
Inhalers are used for emphysema, bronchitis, and asthma. They are used to inhale smoke in the lungs and to help control bronchitis. Inhalers are used for emphysema, bronchitis, and asthma.

⚠️ This is educational only — please consult a qualified clinician.
------------------------------------------------------

Prompt: Describe safe hand-washing steps to prevent infection.
Hand washing should be done at least twice a day and in a gentle, non-abrasive environment.

⚠️ This is educational only — please consult a qualified clinician.
------------------------------------------------------

